In [6]:
from pyhmy import cli
import json
import pandas as pd
import numpy as np
import os
import time
import requests

In [2]:
env = cli.download()
cli.environment.update(env)

Saved harmony binary to: `/Users/yishuang/Desktop/log_analysis/notebooks/staking_dashboard/bin/hmy`


In [3]:
path = os.getcwd() + "/bin/hmy"
cli.set_binary(path)

True

In [4]:
def R1_test():
    committees = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io utility committees"))
    harmony_nodes = []
    for k,v in committees['result']['current']['quorum-deciders'].items():
        for i in v['committee-members']:
            if i['is-harmony-slot'] == True:
                harmony_nodes.append(i['earning-account'])
    
    # no harmony nodes should have validator information
    num = 0
    for i in harmony_nodes:
        try:
            cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i)
        except:
            num += 1
    assert num == len(harmony_nodes)
    return True
            

In [5]:
R1_test()

True

In [11]:
json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain latest-header"))['result']

{'blockHash': '0x4faa57063038531d9ca47264f48bcecf3837c1ff7a91ce0718125e997099ff3f',
 'blockNumber': 16839,
 'epoch': 443,
 'lastCommitBitmap': '9fe27edebf791ff5ffff0f',
 'lastCommitSig': 'e218ba2f2b564843cabd97c3a6b514ce6e611a35f2541bcf4b0f6c21de49f27380db096457dd7f8f156e0fd8c91d9a122ea8ce18033388dedd09fc2e343acc31d0b973aeedf7b5ba884906af1e71a58f337eed26ad80ae616f100f8b68ce4281',
 'leader': 'one18ahxsrk9g4h4gz5r8ema7nyw6g9zpun5hhp54d',
 'shardID': 0,
 'timestamp': '2020-03-23 21:26:37 +0000 UTC',
 'unixtime': 1584998797,
 'viewID': 16839}

In [12]:
def E1_test():
    committees = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io utility committees"))
    slot = committees['result']['current']['external-slot-count']
    validator = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator all"))['result'] 
    epoch = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain latest-header"))['result']['epoch']
    # get top #slots nodes who are eligible to elected next epoch
    new = []
    stake = dict()
    for i in validator:
        info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']
        if info['epos-status'] == 'currently elected and signing enough blocks to be eligible for election next epoch' or info['epos-status'] == 'eligible to be elected next epoch':
            new.append(i)
            stake[i] = info['total-delegation']
    if len(new) > slot:
        sorted_stake = sorted(stake.items(), key=lambda kv: kv[1], reverse = True)
        new = [kv[0] for kv in sorted_stake[:slot]]
    
    # get the lowest stakes among the top #slots
    low = new[-1]
    low_stake = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + low))['result']['total-delegation']
    
    # wait for epoch changes
    new_epoch = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain latest-header"))['result']['epoch']
    while new_epoch == epoch:
        time.sleep(8)
        new_epoch = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain latest-header"))['result']['epoch']
    
    # check whether the eligible validators are selected
    for i in new:
        info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']
        
        if info['epos-status'] == 'eligible to be elected next epoch' or \
        info['epos-status'] == 'not eligible to be elected next epoch':
            print("validator " + i + " who is eligible to be selected is not selected")
            if info['total-delegation'] <= low_stake:
                print("This validator's stake is lower than the lowest stakes in the committee")
            else:
                print("This validator's stake is higher than the lowest stakes in the committee")


In [13]:
E1_test()

validator one164e2dwupqxd7ssr85ncnkx3quk7fexy0eta2vy who is eligible to be selected is not selected
This validator's stake is higher than the lowest stakes in the committee


In [14]:
i = 'one164e2dwupqxd7ssr85ncnkx3quk7fexy0eta2vy'
info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']
info

{'current-epoch-performance': None,
 'currently-in-committee': False,
 'epos-status': 'eligible to be elected next epoch',
 'lifetime': {'apr': '0.000000000000000000',
  'blocks': {'signed': 0, 'to-sign': 0},
  'reward-accumulated': 0},
 'metrics': None,
 'total-delegation': 2.166666e+24,
 'validator': {'address': 'one164e2dwupqxd7ssr85ncnkx3quk7fexy0eta2vy',
  'bls-public-keys': ['2eba33b81d0aacb22e793625077d0c57dd6d38d753bbf39dc720dea0f4a5c056c81d8967a99c2cf1811d0327b049ab16',
   '394d726edbb908fa71bdb2b417dd0fe7c77d9f86b3a0b1e1b51d11e26dd1cddb4e21269c3e1d08e75d43ec98ff4ca582',
   '73ac6379c785142d46bf8423469639fa1dfb9e416e97206cff2c07522a18999a54faaf9cbcad1e34a6c2dc770224848e',
   'b682337f407ec929e7cc18e5103da8a038d63118990b9d8124b31836d8b394a695f16d88b6a32b2543b1dcaaba99a582'],
  'creation-height': 1014,
  'delegations': [{'amount': 2.166666e+24,
    'delegator-address': 'one164e2dwupqxd7ssr85ncnkx3quk7fexy0eta2vy',
    'reward': 0,
    'undelegations': []}],
  'details': 'sentry 

the median stake calcualtion should be applied to all the contested validators, who are eligible for elected next epoch. 
- before election, get the top 200 eligible nodes, and calculate the median;
- after election, calculate the median for the elected node. both the two status:
    - currently elected and signing enough blocks to be eligible for election next epoch 
    - currently elected and not signing enough blocks to be eligible for election next epoch
    
Q: if they are elected, they have the metrics, not elected no metrics?

In [24]:
def M5_test():
    median = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain median-stake"))
    validator = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator all"))['result'] 
    
    # get the median stake and the upper and lower level 
    median_stake = float(median['result']['epos-median-stake'])
    lower = np.round(median_stake - 0.15*median_stake)
    upper = np.round(median_stake + 0.15*median_stake)
    print("median stake is " + median['result']['epos-median-stake'])
    print("lower bond is " + str(lower))
    print("upper bond is " + str(upper))
    print("===========================")
    print("")
    # check every validator's effective stake
    for i in validator:
        info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']
        if info['metrics']:
            for s in info['metrics']['by-shard']:
                if float(s['effective-stake']) > upper or float(s['effective-stake']) < lower:
                    print("validator: " + i + " bls public key: " + str(s['bls-public-key']) +\
                          " shard: " + str(s['shard-id'])) 
                    print("effective stake is out of range. The effective stake is " + s['effective-stake'])
                    print("")

In [25]:
M5_test()

median stake is 297500000000000000000000.000000000000000000
lower bond is 2.52875e+23
upper bond is 3.42125e+23

validator: one103q7qe5t2505lypvltkqtddaef5tzfxwsse4z7 bls public key: 86dc2fdc2ceec18f6923b99fd86a68405c132e1005cf1df72dca75db0adfaeb53d201d66af37916d61f079f34f21fb96 shard: 2
effective stake is out of range. The effective stake is 345000000000000000000000.000000000000000000

validator: one1egemh5e9xjy3x8d3cq0kq7mw4sw4jjwgkc7axs bls public key: d90b4af4db279822edc6b4aa6a2b083512d0b6a873d8e71c814b7994e0f32b97f6a6d81d21344d7fa85e76f76af5e783 shard: 3
effective stake is out of range. The effective stake is 345000000000000000000000.000000000000000000

validator: one12d5a58rcpyf8chlcd2my8r8ns572uppetmxqrx bls public key: 6d5c07ee92b7caff12c245724cc065a8274113ae815c45f257f3c3f81017ab1f401bf76ca11c23f9fe7d0a375245648b shard: 3
effective stake is out of range. The effective stake is 345000000000000000000000.000000000000000000

validator: one17d60e5nvjnzechwl56y38ze6w49wejhtanncra bl

question: in the same epoch, will the delegation change

In [10]:
def get_median(lst):
    n = len(lst) 
    lst.sort() 
    if n % 2 == 0: 
        median1 = lst[n//2] 
        median2 = lst[n//2 - 1] 
        median = (median1 + median2)/2
    else: 
        median = lst[n//2] 
    return median

In [49]:
def M2_test():
    header = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain latest-header"))['result']
    while header['blockNumber'] % 38 != 37:
        time.sleep(8)
        header = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain latest-header"))['result']
    print("current block number: " + str(header['blockNumber']))
    median = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain median-stake"))
    median_stake = float(median['result']['epos-median-stake'])

    elected = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator elected"))
    stake = []
    for i in validator:
        info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']
        print(info)
        print("=====")
        print("")
        stake.append(info['total-delegation'])
    cal_median = get_median(stake)
#     assert median == cal_median
    print("manually calculated median stake: " + str(cal_median))
    print("harmony apr call median stake: " + str(median_stake))

### old E3-test

In [16]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)

In [ ]:
def getAllValidator():
    method = 'hmy_getAllValidatorAddresses'
    params = []
    return get_information(method, params)['result'] 

In [17]:
i = 'one1fd094898rfktel7rezmk65zql3cjzhw9zgxcff'
getValidatorInfo(i)

{'jsonrpc': '2.0',
 'id': 1,
 'result': {'validator': {'bls-public-keys': ['4db5eeb4ff5379f7ba51d522fb17a07571cf427fae86cab31e9927e5048339d7a3c40941b1ce485f75f750aabbf69d82'],
   'last-epoch-in-committee': 861,
   'min-self-delegation': 10000000000000000000000,
   'max-total-delegation': 100000000000000000000000,
   'rate': '0.100000000000000000',
   'max-rate': '0.100000000000000000',
   'max-change-rate': '0.100000000000000000',
   'update-height': 17599,
   'name': 'Yishuang',
   'identity': 'Yishuang',
   'website': 'yishuang@harmony.one',
   'security-contact': 'Yishuang',
   'details': 'Yishuang the validator',
   'creation-height': 17599,
   'address': 'one1fd094898rfktel7rezmk65zql3cjzhw9zgxcff',
   'delegations': [{'delegator-address': 'one1fd094898rfktel7rezmk65zql3cjzhw9zgxcff',
     'amount': 10000000000000000000000,
     'reward': 16772603128884595638474,
     'undelegations': []}]},
  'current-epoch-performance': None,
  'metrics': None,
  'total-delegation': 100000000000

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [13]:
# it doesn't work
def getAllValidatorByBlock(block):
    method = "hmy_getAllValidatorInformationByBlockNumber"
    params = [0, block]
    return get_information(method, params)

In [14]:
block = getBlockNumber()
block

32777

In [15]:
getAllValidatorByBlock(hex(block))

{'jsonrpc': '2.0',
 'id': 1,
 'result': [{'validator': {'bls-public-keys': ['afc4d81ab636323b04893c23d3404f38e95e9291b3d5299dade3d03822edd3a2b799bd6d64265a9a9cac025d5dbd600c'],
    'last-epoch-in-committee': 862,
    'min-self-delegation': 10000000000000000000000,
    'max-total-delegation': 100000000000000000000000,
    'rate': '0.050000000000000000',
    'max-rate': '0.800000000000000000',
    'max-change-rate': '0.020000000000000000',
    'update-height': 131,
    'name': 'OSTNSentry0',
    'identity': 'OSTNSentry0',
    'website': 'sentry@harmony.one',
    'security-contact': 'Sentry',
    'details': 'OSTN Sentry for Shard 0 t3.small',
    'creation-height': 131,
    'address': 'one1m6j80t6rhc3ypaumtsfmqwjwp0mrqk9ff50prh',
    'delegations': [{'delegator-address': 'one1m6j80t6rhc3ypaumtsfmqwjwp0mrqk9ff50prh',
      'amount': 100000000000000000000000,
      'reward': 50060816679307776641887,
      'undelegations': []}]},
   'current-epoch-performance': {'current-epoch-signing-percen

In [ ]:
def E3_test():
    # get the first block in current epoch
    block = getBlockNumber()
    while block % 38 != 0:
        time.sleep(8)
        block = getBlockNumber()
    print("Starting the first block in current epoch")
    val_old = getAllValidator()

    # get the second to last block in current epoch
    block = getBlockNumber()
    while block % 38 != 36:
        time.sleep(8)
        block = getBlockNumber()
    print("Starting the second to last block in current epoch")
    val_new = getAllValidator()

    # get all the new nodes joining after the election
    new = [i for i in val_new if i not in val_old]
    
    # get the hex of the first block in next epoch
    if getBlockNumber() < block+3:
        time.sleep(8)
    
    if not new:
        print("no new nodes joined before the next election start")
        
    hex_block = hex(block+2)
    for i in new:
        print("validator " + i + " joined before the next election start: ")
        status = getValidatorByBlock(i, hex_block)['epos-status']
        print(status)
        if status == 'currently elected and not signing enough blocks to be eligible for election next epoch' \
        or status == 'not eligible to be elected next epoch':
            print("[FAILED] Joining before the election was not considered as the validator")
            return False
    return True